In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import xarray as xr
import intake
import pandas as pd
from pyremo.preproc import era5

In [11]:
from dask.distributed import Client, progress
#client = Client(n_workers=6, threads_per_worker=4)
client = Client()
client

/work/ch0636/g300046/conda_envs/cmip6-processing/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42801 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/g300046/mistral-spawner-advanced//proxy/42801/status,
Dashboard: /user/g300046/mistral-spawner-advanced//proxy/42801/status,Workers: 6
Total threads: 24,Total memory: 58.59 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37195,Workers: 6
Dashboard: /user/g300046/mistral-spawner-advanced//proxy/42801/status,Total threads: 24
Started: Just now,Total memory: 58.59 GiB
Comm: tcp://127.0.0.1:37675,Total threads: 4
Dashboard: /user/g300046/mistral-spawner-advanced//proxy/43572/status,Memory: 9.77 GiB
Nanny: tcp://127.0.0.1:33381,


In [133]:
df_era5 = pd.read_csv('/mnt/lustre01/work/kd0956/Catalogs/mistral-era5.csv.gz')

/work/ch0636/g300046/conda_envs/cmip6-processing/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3012: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [81]:
df_era5.head()

,era_id,dataType,level_type,frequency,stepType,code,short_name,long_name,units,validation_date,initialization_date,step,path
0,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-01,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...
1,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-02,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...
2,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-03,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...
3,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-04,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...
4,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-05,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...


In [82]:
era51 = "/pool/data/ERA5/*/*/*era5.1*/*"

In [134]:
from glob import glob
from pathlib import Path
import parse
import pandas as pd
files = glob(era51)
files.sort()

In [135]:
filename = '/pool/data/ERA5/ml00_1H/2000/01-era5.1/E1ml00_1H_2000-01_075.grb'
pattern = "{era_id:2}{level_type:2}00_{frequency:2}_{validation_date:6}_{code:3d}"

In [136]:
freqs = {'1H': 'hourly', '1D': 'daily'}
levels = {'ml': 'model_level', 'pl': 'pressure_level', 'sf': "surface"}


def parse_file(file, pattern):
    file = Path(file).stem
    finfo = parse.parse(pattern, file)
    #if finfo is not None:
    return finfo.named

def rename(data):
    data['frequency'] = freqs[data['frequency']]
    data['level_type'] = levels[data['level_type']]
    return data
    
    
def parse_files(files, pattern):
    results = []
    for f in files:
        data = parse_file(f, pattern)
        data["path"] = f
        data = rename(data)
        results.append(data)
    return results
   

In [137]:
data = parse_files(files, pattern)

In [138]:
df_era51 = pd.DataFrame(data)
df_era51.head()

,era_id,level_type,frequency,validation_date,code,path
0,E1,model_level,hourly,2000-01,75,/pool/data/ERA5/ml00_1H/2000/01-era5.1/E1ml00_...
1,E1,model_level,hourly,2000-01,76,/pool/data/ERA5/ml00_1H/2000/01-era5.1/E1ml00_...
2,E1,model_level,hourly,2000-01,77,/pool/data/ERA5/ml00_1H/2000/01-era5.1/E1ml00_...
3,E1,model_level,hourly,2000-01,129,/pool/data/ERA5/ml00_1H/2000/01-era5.1/E1ml00_...
4,E1,model_level,hourly,2000-01,130,/pool/data/ERA5/ml00_1H/2000/01-era5.1/E1ml00_...


In [139]:
df_era51.level_type.unique()

array(['model_level', 'pressure_level'], dtype=object)

In [92]:
df_era5[(df_era5.code == 130) & (df_era5.level_type == "model_level") & (df_era5.validation_date == "2000-01-01")]

,era_id,dataType,level_type,frequency,stepType,code,short_name,long_name,units,validation_date,initialization_date,step,path
69290,E5,an,model_level,hourly,instant,130,t,Temperature,K,2000-01-01,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/2000/E5...


In [140]:
def update_table(df1, df2):
    codes = df2.code.unique()
    df_drop = df1.drop(df1.loc[df1['code'].isin(codes)].index)
    return pd.concat([df_drop, df2], ignore_index=True)
    

#df.drop(df.loc[df['code']==130].index)

In [141]:
df = update_table(df_era5, df_era51)
#df = pd.concat([df_era5, df_era51], ignore_index=True)
#df.head()

In [105]:
df.head()

,era_id,dataType,level_type,frequency,stepType,code,short_name,long_name,units,validation_date,initialization_date,step,path
0,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-01,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...
1,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-02,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...
2,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-03,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...
3,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-04,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...
4,E5,an,model_level,hourly,instant,75,crwc,Specific rain water content,kg kg**-1,1979-01-05,NaN,NaN,/mnt/lustre02/work/bk1099/data/ml00_1H/1979/E5...


In [119]:
df[(df.code == 34) & (df.level_type == "surface") & (df.validation_date.isin(["2000-01"]))]

,era_id,dataType,level_type,frequency,stepType,code,short_name,long_name,units,validation_date,initialization_date,step,path
627246,E5,an,surface,hourly,instant,34,sst,Sea surface temperature,K,2000-01,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/2000/E5...


In [ ]:
df

In [142]:
cmor = era5.ERA5(df=df)

In [131]:
import pandas as pd
from datetime import datetime
datelist = pd.date_range("2000-01-01T00:00:00", "2000-01-02T00:00:00", freq='6H').strftime('%Y-%m-%dT%H:%M:%S').tolist()
datelist

['2000-01-01T00:00:00',
 '2000-01-01T06:00:00',
 '2000-01-01T12:00:00',
 '2000-01-01T18:00:00',
 '2000-01-02T00:00:00']

In [152]:
era5.show_cdo=True

In [150]:
sel = era5._get_row_by_date(131, "2001-01-01T00:00:00", cmor.df)
sel

era_id                                                                E1
dataType                                                             NaN
level_type                                                   model_level
frequency                                                         hourly
stepType                                                             NaN
code                                                                 131
short_name                                                           NaN
long_name                                                            NaN
units                                                                NaN
initialization_date                                                  NaN
step                                                                 NaN
path                   /pool/data/ERA5/ml00_1H/2001/01-era5.1/E1ml00_...
time                                                 2001-01-01 00:00:00
Name: 2001-01, dtype: object

In [128]:
df = cmor.df
df[(df.code == 34) & (df.level_type == "surface") & (df.frequency == "hourly")]#.loc["2000-01-01"]#.path.unique()

,era_id,dataType,level_type,frequency,stepType,code,short_name,long_name,units,initialization_date,step,path,time
validation_date,,,,,,,,,,,,,
1979-01,E5,an,surface,hourly,instant,34,sst,Sea surface temperature,K,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...,1979-01-01
1979-02,E5,an,surface,hourly,instant,34,sst,Sea surface temperature,K,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...,1979-02-01
1979-03,E5,an,surface,hourly,instant,34,sst,Sea surface temperature,K,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...,1979-03-01
1979-04,E5,an,surface,hourly,instant,34,sst,Sea surface temperature,K,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...,1979-04-01
1979-05,E5,an,surface,hourly,instant,34,sst,Sea surface temperature,K,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...,1979-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,E5,fc,surface,hourly,instant,34,sst,Sea surface temperature,K,2020-10,1.0,/mnt/lustre02/work/bk1099/data/sf12_01/2020/E5...,NaT
NaN,E5,fc,surface,hourly,instant,34,sst,Sea surface temperature,K,2020-11,1.0,/mnt/lustre02/work/bk1099/data/sf12_01/2020/E5...,NaT
NaN,E5,fc,surface,hourly,instant,34,sst,Sea surface temperature,K,2020-12,1.0,/mnt/lustre02/work/bk1099/data/sf12_01/2020/E5...,NaT


In [153]:
cmor.to_xarray('ps', "2000-01-01T00:00:00")

cdo  seldate,2000-01-01T00:00:00 /mnt/lustre02/work/bk1099/data/sf00_1H/2000/E5sf00_1H_2000-01_134 /scratch/g/g300046/.cdo_tmp/tmpjsltqt17
cdo  griddes /scratch/g/g300046/.cdo_tmp/tmpjsltqt17 
cdo -f nc  setgridtype,regular /scratch/g/g300046/.cdo_tmp/tmpjsltqt17 /scratch/g/g300046/.cdo_tmp/tmp9it8ar7_
cdo  invertlat /scratch/g/g300046/.cdo_tmp/tmp9it8ar7_ /scratch/g/g300046/.cdo_tmp/tmpwsilpgfn


<xarray.Dataset>
Dimensions:  (time: 1, lon: 1280, lat: 640)
Coordinates:
  * time     (time) object 2000-01-01 00:00:00
  * lon      (lon) float64 0.0 0.2812 0.5625 0.8438 ... 358.9 359.2 359.4 359.7
  * lat      (lat) float64 -89.78 -89.51 -89.23 -88.95 ... 89.23 89.51 89.78
Data variables:
    ps       (time, lat, lon) float32 dask.array<chunksize=(1, 640, 1280), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.9 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Tue Apr 19 16:50:54 2022: cdo invertlat /scratch/g/g300046/...
    CDO:          Climate Data Operators version 1.9.9 (https://mpimet.mpg.de...

In [98]:
sel.path

'/mnt/lustre02/work/bk1099/data/sf00_1H/2000/E5sf00_1H_2000-01_134'

In [181]:
gfile = cmor.gfile(datelist[0], parallel=False, add_fx=True, fx_date="2000-01-01T00:00:00")

100%|██████████| 1/1 [02:06<00:00, 126.32s/it]


In [182]:
gfile

<xarray.Dataset>
Dimensions:   (lat: 640, time: 1, lon: 1280, bnds: 2, lev: 137, nhyi: 138, nhym: 137)
Coordinates:
  * lat       (lat) float64 -89.78 -89.51 -89.23 -88.95 ... 89.23 89.51 89.78
  * time      (time) object 2000-01-01 00:00:00
  * lon       (lon) float64 0.0 0.2812 0.5625 0.8438 ... 358.9 359.2 359.4 359.7
  * lev       (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 134.0 135.0 136.0 137.0
Dimensions without coordinates: bnds, nhyi, nhym
Data variables: (12/18)
    lat_bnds  (time, lat, bnds) float64 dask.array<chunksize=(1, 640, 2), meta=np.ndarray>
    hyai      (nhyi) float64 dask.array<chunksize=(138,), meta=np.ndarray>
    hybi      (nhyi) float64 dask.array<chunksize=(138,), meta=np.ndarray>
    hyam      (nhym) float64 dask.array<chunksize=(137,), meta=np.ndarray>
    hybm      (nhym) float64 dask.array<chunksize=(137,), meta=np.ndarray>
    ta        (time, lev, lat, lon) float32 dask.array<chunksize=(1, 137, 640, 1280), meta=np.ndarray>
    ...        ...
    va        (time, lev, lat, lon) float32 dask.array<chunksize=(1, 137, 640, 1280), meta=np.ndarray>
    ua        (time, lev, lat, lon) float32 dask.array<chunksize=(1, 137, 640, 1280), meta=np.ndarray>
    orog      (lat, lon) float32 dask.array<chunksize=(640, 1280), meta=np.ndarray>
    sftlf     (lat, lon) float32 dask.array<chunksize=(640, 1280), meta=np.ndarray>
    akgm      (nhyi) float64 dask.array<chunksize=(138,), meta=np.ndarray>
    bkgm      (nhyi) float64 dask.array<chunksize=(138,), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.9 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Tue Apr 19 17:36:35 2022: cdo invertlat /scratch/g/g300046/...
    CDO:          Climate Data Operators version 1.9.9 (https://mpimet.mpg.de...

In [179]:
era5.show_cdo=True
fx = cmor._fx(date="2000-01-01T00:00:00")

cdo  seldate,2000-01-01T00:00:00 /pool/data/ERA5/ml00_1H/2000/01-era5.1/E1ml00_1H_2000-01_129.grb /scratch/g/g300046/.cdo_tmp/tmpppx9345o
cdo  griddes /scratch/g/g300046/.cdo_tmp/tmpppx9345o 
cdo -f nc  sp2gpl /scratch/g/g300046/.cdo_tmp/tmpppx9345o /scratch/g/g300046/.cdo_tmp/tmpc9cgo567
cdo  invertlat /scratch/g/g300046/.cdo_tmp/tmpc9cgo567 /scratch/g/g300046/.cdo_tmp/tmppdjo9daq
cdo  seldate,2000-01-01T00:00:00 /mnt/lustre02/work/bk1099/data/sf00_1H/2000/E5sf00_1H_2000-01_172 /scratch/g/g300046/.cdo_tmp/tmpzrwj1mfh
cdo  griddes /scratch/g/g300046/.cdo_tmp/tmpzrwj1mfh 
cdo -f nc  setgridtype,regular /scratch/g/g300046/.cdo_tmp/tmpzrwj1mfh /scratch/g/g300046/.cdo_tmp/tmpu0j6qn2e
cdo  invertlat /scratch/g/g300046/.cdo_tmp/tmpu0j6qn2e /scratch/g/g300046/.cdo_tmp/tmpaauv5ph5


In [180]:
fx

<xarray.Dataset>
Dimensions:   (lat: 640, lon: 1280, bnds: 2, nhyi: 138, nhym: 137)
Coordinates:
  * lat       (lat) float64 -89.78 -89.51 -89.23 -88.95 ... 89.23 89.51 89.78
  * lon       (lon) float64 0.0 0.2812 0.5625 0.8438 ... 358.9 359.2 359.4 359.7
    lev       float64 1.0
Dimensions without coordinates: bnds, nhyi, nhym
Data variables:
    lat_bnds  (lat, bnds) float64 dask.array<chunksize=(640, 2), meta=np.ndarray>
    hyai      (nhyi) float64 dask.array<chunksize=(138,), meta=np.ndarray>
    hybi      (nhyi) float64 dask.array<chunksize=(138,), meta=np.ndarray>
    hyam      (nhym) float64 dask.array<chunksize=(137,), meta=np.ndarray>
    hybm      (nhym) float64 dask.array<chunksize=(137,), meta=np.ndarray>
    orog      (lat, lon) float32 dask.array<chunksize=(640, 1280), meta=np.ndarray>
    sftlf     (lat, lon) float32 dask.array<chunksize=(640, 1280), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.9 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Tue Apr 19 17:07:50 2022: cdo invertlat /scratch/g/g300046/...
    CDO:          Climate Data Operators version 1.9.9 (https://mpimet.mpg.de...

In [74]:
write = to_netcdf(gfile, path="/scratch/g/g300046/ERA5/", expid='056000', template='a{}g{}.nc')